In [21]:
import pandas as pd
from pymongo import MongoClient
import datetime
import time
from datetime import date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import pymongo
import json
from matplotlib.backends.backend_pdf import PdfPages
import os
import csv
from pandas import read_csv
import numpy as np
import random

In [25]:
df_complete=pd.read_csv("/home/seil/karanj/git_seil_repo1/dataset/virtual_dataset/complete.csv")
df_complete['Apartment']=df_complete['Virtual_Apartment']
for col in list(df_complete):
    if col not in ['day','time','Apartment','power']:
        df_complete=df_complete.drop(col,1)
df_complete.to_csv('building_data.csv',index=False)

In [29]:
df=pd.read_csv("building_data.csv")
df.head(10)

,day,time,power,Apartment
0,1,07:30:00,0.0,5
1,1,08:30:00,3.1287042045591664,5
2,1,09:30:00,3.8517009417218344,5
3,1,10:30:00,4.862898861566666,5
4,1,11:30:00,4.730838634173167,5
5,1,12:30:00,6.0299455674489995,5
6,1,13:30:00,11.167015473044836,5
7,1,14:30:00,10.809621089305335,5
8,1,15:30:00,12.705798606875831,5
9,1,16:30:00,4.766880884170833,5


In [32]:
df=pd.read_csv("building_data.csv")
t=df.Apartment.unique()
apartments=t.tolist()

day_0 = df [df['day']==1] 
day_1 = df [df['day']==2] 
day_2 = df [df['day']==3] 
day_3 = df [df['day']==4] 
day_4 = df [df['day']==5] 

day_5 = df [df['day']==6] 
day_6 = df [df['day']==7] 
day_7 = df [df['day']==8] 
day_8 = df [df['day']==9] 
day_9 = df [df['day']==10]  


values= df.drop('date',1)
values= values.drop('power',1)
values=values.drop_duplicates()
values=values.reset_index(drop=True)
total_dr=90

values['max_request']=5
values['total_DR']=total_dr
values['requests']=np.random.randint(40,90,values.shape[0])
values['temp']=np.random.randint(1,100,values.shape[0]) 
values['response']=(values['requests']*values['temp'] )//100
values['count']=0

values['req_index']=(values['requests']*100)//values['total_DR']
values['res_index']=(values['response']*100)//values['requests']
values['rel_index']=(values['res_index'] + values['cons'])//2
values['contri_index']=0
values['inconv']=0

#converting to percentile
values['req_p']=(values['req_index']*100)/(max(values['req_index']))
values['res_p']=(values['res_index']*100)/(max(values['res_index']))
values['rel_p']=(values['rel_index']*100)/(max(values['rel_index']))
values['contri_p']=(values['contri_index']*100)/(max(values['contri_index']))
values=values.fillna(0)

values['score']=values['contri_p']+values['rel_p']+values['res_p']+values['req_p']
values['score']=values['score']/4 #score out of 100

values=values.drop('temp',1)

values.to_csv("values.csv")
print (values.shape)

(500, 17)


In [33]:
values['cons']=np.random.randint(50,80,values.shape[0])
for xyz in range(10):
    

    values['max_request']=5
    values['total_DR']=total_dr
    values['requests']=np.random.randint(40,90,values.shape[0])
    values['temp']=np.random.randint(1,100,values.shape[0]) 
    values['response']=(values['requests']*values['temp'] )//100
    values['count']=0


    values['req_index']=(values['requests']*100)//values['total_DR']
    values['res_index']=(values['response']*100)//values['requests']
    values['rel_index']=(values['res_index'] + values['cons'])//2
    values['contri_index']=0
    values['inconv']=0

    #converting to percentile
    values['req_p']=(values['req_index']*100)/(max(values['req_index']))
    values['res_p']=(values['res_index']*100)/(max(values['res_index']))
    values['rel_p']=(values['rel_index']*100)/(max(values['rel_index']))
    values['contri_p']=(values['contri_index']*100)/(max(values['contri_index']))
    values=values.fillna(0)

    values['score']=values['contri_p']+values['rel_p']+values['res_p']+values['req_p']
    values['score']=values['score']/4 #score out of 100

    values=values.drop('temp',1)

    values.to_csv("values_1.csv")


    from time import gmtime, strftime
    print (strftime("%Y-%m-%d %H:%M:%S"))

    for it in range(1,16):
        print (it,end='')

        values=pd.read_csv("values_1.csv")
        days=[day_0,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9]
        benefit =[]
        damage=0
        incov =[]

        x=5
        t_dr=90
        ttt=0

        for day in days:
    #         print "Day "
            t_dr=t_dr+1
            #calc contribution factor
            result= day.drop('Apartment',1)
            result=result.groupby(df['date'])['power'].agg(['sum','mean','max'])
    #         result.plot()
            result.to_csv("day_agg.csv")
            result=pd.read_csv("day_agg.csv")
            
            day= pd.merge(day,result,how='left',on=['date'])
    #         print "Merge 1"
            #day.loc[day.Apartment<=125,"power"]=day.loc[day.Apartment<=125,"power"]*4

            day["contribution"]=(day["power"]/day["sum"])*100
            day["contribution_index"] = day["contribution"] *100 / max(day["contribution"]) 


            values['req_index']=( (values['total_DR'] - values['requests'])*100)//values['total_DR']
            values['res_index']=(values['response']*100)//values['requests']
            values['rel_index']=(values['res_index'] + values['cons'])//2

            values['cons_p']=(values['cons']*100)/(max(values['cons']))
            values['req_p']=(values['req_index']*100)/(max(values['req_index']))
            values['res_p']=(values['res_index']*100)/(max(values['res_index']))


            values['inconv']=0
            day= pd.merge(day,values,how='left',on=['Apartment'])
    #         print "Merge 2"
            day["contri_p"]=day["contribution_index"]

            if it ==1 : 
                day['score']=day['cons_p']
            if it==2 : 
                day['score']=day['contri_p']
            if it ==3 :
                day['score']=day['req_p']     
            if it ==4 :
                day['score']=day['res_p']


            if it ==5 :
                day['score']=day['cons_p']+day['contri_p']
            if it ==6 :
                day['score']=day['cons_p']+day['req_p']
            if it ==7 :
                day['score']=day['cons_p']+day['res_p']

            if it ==8 :
                day['score']=day['contri_p']+day['req_p']
            if it ==9 :
                day['score']=day['contri_p']+day['res_p']

            if it ==10 :
                day['score']=day['res_p']+day['req_p']


            if it ==11 :
                day['score']=day['cons_p']+day['contri_p']+day['req_p']
            if it ==12 :
                day['score']=day['cons_p']+day['contri_p']+day['res_p']
            if it ==13 :
                day['score']=day['cons_p']+day['req_p']+day['res_p']            
            if it ==14 :
                day['score']=day['contri_p']+day['req_p']+day['res_p']


            if it ==15 :
                day['score']=day['cons_p']+day['contri_p']+day['req_p']+day['res_p']



            to_reduce=day["sum"].unique()[0] #result[(result.date=='2017-06-06 17:00:00')]["max"]
        #     print "Reducing " + str(0.2*to_reduce) + " of " +str(to_reduce)
            to_reduce=0.2*to_reduce
            tr=to_reduce

            day=day.sort_values(['score'],ascending=False)

            apartment_count=0
            ap=[]

    #         print "Choosing"
            for index,row in day.iterrows():  
                apartment_number=row["Apartment"]
                inconv=0
                maxi = values[values['Apartment']==apartment_number]['max_request'].as_matrix()[0]
                c = values[values['Apartment']==apartment_number]['count'].as_matrix()[0]

                #inits
                requests_x=row['requests']
                response_x=row['response']
                count_x=row['count']


                if (int(maxi)!=int(c)) and (to_reduce >0):
                    ap.append(row["Apartment"])
                    benefit.append(row["Apartment"])
                    apartment_count=apartment_count+1
                    to_reduce=to_reduce-0.5*row['power']
                    requests_x=row['requests']+1
                    response_x=row['response']+ 1
                    count_x=row['count']+ 1
                    inconv=row['contribution']
                    damage= damage+ count_x*inconv

                    values.loc[values['Apartment']==apartment_number,'requests']=requests_x
                    values.loc[values['Apartment']==apartment_number,'response']=response_x
                    values.loc[values['Apartment']==apartment_number,'count']=count_x
                    values.loc[values['Apartment']==apartment_number,'inconv']=inconv


                contri_index_x=row['contribution']
                contri_p_x=row['contribution_index']
                requests_index_x=(row['requests']*100)//row['total_DR']
                response_index_x=(row['response']*100)//row['requests']
                reliablity_index_x=(row['res_index'] + row['cons'])//2


                values.loc[values['Apartment']==apartment_number,'req_index']=requests_index_x
                values.loc[values['Apartment']==apartment_number,'res_index']=response_index_x
                values.loc[values['Apartment']==apartment_number,'rel_index']=reliablity_index_x
                values.loc[values['Apartment']==apartment_number,'contri_index']=contri_index_x
                values.loc[values['Apartment']==apartment_number,'contri_p']=contri_p_x
                values.loc[values['Apartment']==apartment_number,'total_DR']=t_dr

            incov.append(sum(values["inconv"]))
            values['score']=values['contri_p']+values['rel_p']+values['res_p']+values['req_p']
            values['score']=values['score']/4 #score out of 100
            values.to_csv("days/day"+str(x)+".csv")

            ttt=ttt+apartment_count
            x=x+1

        org=pd.read_csv("values_1.csv")
        risk=0
        for elem in benefit:
            risk = risk + (100- org[org['Apartment']==elem]['rel_index'].as_matrix()[0] )

        print (","+str(risk/len(benefit))+"," +str( (500- len(set(benefit))) )+"," +str(ttt))

    #     print "Risk " + str(risk/len(benefit))
    #     print "Unfairness " + str(25-len(set(benefit))) #+ "    " +   str(set(benefit))   
    #     print "Inconv " + str(damage)
    #     print "Total DR sent "+str(ttt)
    #     print "***********************************************"

    print (strftime("%Y-%m-%d %H:%M:%S"))


2018-06-01 02:11:04
1

/home/seil/.local/lib/python3.5/site-packages/ipykernel_launcher.py:154: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/seil/.local/lib/python3.5/site-packages/ipykernel_launcher.py:155: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/seil/.local/lib/python3.5/site-packages/ipykernel_launcher.py:205: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


,42.99631190727081,107,1898
2,42.55028462998102,182,1054
3,44.5715700842001,73,2019
4,39.77325876970005,100,1967
5,41.65505693235097,135,1493
6,43.90125944584383,86,1985
7,39.345146379044685,104,1947
8,44.47478753541076,101,1765
9,37.39464493597206,128,1718
10,40.72508942258559,99,1957
11,43.617332583005066,105,1777
12,37.42168674698795,123,1743
13,40.423469387755105,100,1960
14,39.38522727272727,121,1760
15,38.99087799315849,124,1754
2018-06-01 02:27:18
2018-06-01 02:27:18
1,41.71707060063225,107,1898
2,43.77703984819734,182,1054
3,43.30112474437628,85,1956
4,38.952,85,2000
5,40.635632953784324,135,1493
6,42.40474961280331,91,1937
7,38.92857142857143,86,2016
8,43.08365867261573,100,1793
9,37.188429752066114,104,1815
10,40.08440276406713,84,2026
11,42.38041643218908,103,1777
12,36.79507278835386,107,1786
13,39.616957605985036,88,2005
14,39.105995717344754,97,1868
15,38.44626295690126,106,1833
2018-06-01 02:43:37
2018-06-01 02:43:37
1,41.79978925184405,107,1898
2,44.202087286527515,182,